In [4]:
import river
import river.datasets as datasets
from river import stream
from pprint import pprint
import os
from scipy.io import arff
import pandas as pd
from river import tree
from river import evaluate
from river import metrics
from semisupervised_methods.clustream_and_label import  CluStreamMicroClusterWithLabel,CluserAndLabel
from river.forest.adaptive_random_forest import ARFClassifier
from semisupervised_methods.incremental_classifier import IncrementalClassifer
import numpy as np
from  streams.stream_section import StreamSection
from streams.utils import FL, FU, generate_stream_section
from my_datasets.getting_datasets import get_LED,get_Airlines
from train_and_eval import train_and_evaluate




In [2]:
lda = get_Airlines()

['.git', '.gitignore', 'constants.py', 'my_datasets', 'README.md', 'semisupervised_methods', 'streams', 'tries.ipynb', '__pycache__']


In [2]:
rbf = datasets.synth.RandomRBFDrift()

In [ ]:
datasets.synth.Hyperplane()

In [3]:
lda = datasets.synth.LEDDrift(123,0,True,0)

In [4]:
ls = StreamSection('LDA_full',list(lda.take(1000)),True)
q = StreamSection('LDA_semi',FU(ls.stream,0.5),False)
nq = StreamSection('LDA_semi_removed',FL( q.stream),True)

In [ ]:
led_stream = datasets.synth.LED(123)
arfclassifier = IncrementalClassifer(0.67,ARFClassifier,3,{'n_models':1})
#arfclassifier = ARFClassifier()
for i,(x,y) in enumerate(q.stream):
    if i <990:
        arfclassifier = arfclassifier.learn_one(x = x,y=y)
        #print(clustream.sum_labels())

    else:
        print(x,arfclassifier.predict_one(x),y)
        print(arfclassifier.predict_proba_one(x))

0 3 None
0 3 None
0 3 2
1 3 None
1 3 5
2 3 7
3 3 8
4 3 None
5 3 None
6 3 7
7 3 7
8 3 8
9 3 None
10 3 None
11 3 6
12 3 8
13 3 9
14 3 None
15 3 None
16 3 4
17 3 3
18 3 None
19 3 9
20 3 None
21 3 6
22 3 None
23 3 None
24 3 None
25 3 None
26 3 9
27 3 None
28 3 6
29 3 8
30 3 None
31 3 7
32 3 None
33 3 2
34 3 None
35 3 None
36 3 9
37 3 3
38 3 3
39 3 0
40 3 None
41 3 None
42 3 None
43 3 None
44 3 2
45 3 2
46 3 None
47 3 0
48 3 9
49 3 None
50 3 0
51 3 None
52 3 None
53 3 None
54 3 None
55 3 5
56 3 8
57 3 0
58 3 None
59 3 2
60 3 3
61 3 None
62 3 3
63 3 None
64 3 2
65 3 1
66 3 2
67 3 5
68 3 1
69 3 None
70 3 None
71 3 3
72 3 9
73 3 4
74 3 None
75 3 None
76 3 None
77 3 9
78 3 3
79 3 None
80 3 None
81 3 2
82 3 6
83 3 None
84 3 4
85 3 None
86 3 None
87 3 2
88 3 None
89 3 None
90 3 None
91 3 6
92 3 2
93 3 None
94 3 9
95 3 None
96 3 None
97 3 1
98 3 None
99 3 None
100 3 None
101 3 4
102 3 None
103 3 None
104 3 3
105 3 None
106 3 5
107 3 1
108 3 2
109 3 None
110 3 None
111 3 None
112 3 7
113 3 0
114 3 

In [8]:
from river import cluster
from river import stream
# X = [ [1, 2],[1, 4],[1, 0],[-4, 0],[-4, 4]]

# Y = [1,1,2,3,1]
clustream = CluserAndLabel(4,
         max_micro_clusters=7,
         time_gap=3,
         seed=0,
         halflife=0.4 )
led_stream = datasets.synth.LED(123)
for i,(x,y) in enumerate(q.stream):
    if i <99:
        # if y ==7:
        #     print(clustream.return_microclusters() )
        if i %10 == 9:
            clustream = clustream.learn_one(x)
        else:
            clustream = clustream.learn_one(x = x,y = y)
        #print(clustream.sum_labels())

    else:
        print(x,clustream.predict_one(x),y)

clustream.return_microclusters() 

{0: 1, 1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 1, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1, 15: 1, 16: 0, 17: 0, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1} 8 7
{0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0} 8 None
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 1, 16: 1, 17: 1, 18: 0, 19: 1, 20: 1, 21: 0, 22: 0, 23: 1} 8 None
{0: 1, 1: 0, 2: 1, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 1, 12: 0, 13: 1, 14: 1, 15: 0, 16: 0, 17: 1, 18: 0, 19: 1, 20: 1, 21: 0, 22: 1, 23: 0} 8 3
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 1, 15: 0, 16: 1, 17: 1, 18: 0, 19: 1, 20: 0, 21: 1, 22: 1, 23: 1} 8 2
{0: 0, 1: 1, 2: 1, 3: 1, 4: 0, 5: 1, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 1, 15: 1, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 1, 22: 1, 23: 0} 8 None
{0: 1, 1: 1, 

# DATASETS

In [2]:
fligths = [(dict((i,x[i]) for i in x.keys() if i!='Delay'),x['Delay']) for x,_ in stream.iter_arff('../../datasets/airlines.arff.zip', compression='infer')]
lda = datasets.synth.LEDDrift(123,0,True,0)
electricity = [(dict((i,x[i]) for i in x.keys() if i!='class'),x['class']=='UP') for x,_ in stream.iter_arff('../../datasets/elecNormNew.arff.zip', compression='infer')]

In [77]:
datasets.synth.LEDDrift()

In [5]:
Q = [(0,1000),(1000,2000),(2000,3000),(3000,4000),(4000,5000)]
datastream = lda
P = [0.1,0.2,0.5,0.6]
dataset_name = 'LED'
stream_set = []

train_and_evaluate(datastream,dataset_name,Q,P)

In [86]:
[x.__name__ for x in stream_set]

['Cover_Type_init_0_1000',
 'Cover_Type_ssl_0.1_0_1000',
 'Cover_Type_lfs_0.1_0_1000',
 'Cover_Type_ssl_0.2_0_1000',
 'Cover_Type_lfs_0.2_0_1000',
 'Cover_Type_ssl_0.5_0_1000',
 'Cover_Type_lfs_0.5_0_1000',
 'Cover_Type_ssl_0.6_0_1000',
 'Cover_Type_lfs_0.6_0_1000',
 'Cover_Type_init_1000_2000',
 'Cover_Type_ssl_0.1_1000_2000',
 'Cover_Type_lfs_0.1_1000_2000',
 'Cover_Type_ssl_0.2_1000_2000',
 'Cover_Type_lfs_0.2_1000_2000',
 'Cover_Type_ssl_0.5_1000_2000',
 'Cover_Type_lfs_0.5_1000_2000',
 'Cover_Type_ssl_0.6_1000_2000',
 'Cover_Type_lfs_0.6_1000_2000',
 'Cover_Type_init_2000_3000',
 'Cover_Type_ssl_0.1_2000_3000',
 'Cover_Type_lfs_0.1_2000_3000',
 'Cover_Type_ssl_0.2_2000_3000',
 'Cover_Type_lfs_0.2_2000_3000',
 'Cover_Type_ssl_0.5_2000_3000',
 'Cover_Type_lfs_0.5_2000_3000',
 'Cover_Type_ssl_0.6_2000_3000',
 'Cover_Type_lfs_0.6_2000_3000',
 'Cover_Type_init_3000_4000',
 'Cover_Type_ssl_0.1_3000_4000',
 'Cover_Type_lfs_0.1_3000_4000',
 'Cover_Type_ssl_0.2_3000_4000',
 'Cover_Type_lfs

In [7]:
for q in stream_set:
    clustream = CluserAndLabel(4,
         max_micro_clusters=7,
         time_gap=3,
         seed=0,
         halflife=0.4 )
    for i,(x,y) in enumerate(q.stream):
        print(q.__name__)
        if i <990:
            clustream = clustream.learn_one(x = x,y = y)
            #print(clustream.sum_labels())

        else:
            print(clustream.predict_one(x),y)

    #clustream.return_microclusters() 

LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init_0_1000
LED_init

KeyboardInterrupt: 

In [70]:
i

999

In [50]:
hp = datasets.synth.RandomRBFDrift()
for i in hp.take(10):
    print(i)

({0: 0.4474707370607147, 1: 0.4112430647339043, 2: 0.5774887345532604, 3: 0.9345089494963857, 4: 0.625470816542423, 5: 0.9528893392318104, 6: 0.23390913685064327, 7: 0.2632706539603502, 8: 0.6487674488470405, 9: 0.3223666205525}, 0)
({0: 0.38615946253321076, 1: 0.40767762106825467, 2: 0.8820781268130979, 3: 1.0294262479340588, 4: 0.16964208338080689, 5: 0.4068516375090169, 6: 0.920811978379108, 7: 0.31470618207032774, 8: 0.2904818353582927, 9: 0.5318226376467426}, 1)
({0: 0.7407199640645749, 1: 0.675852803052017, 2: 0.09199640007669287, 3: 0.6597045689218738, 4: 0.9128605755224982, 5: 0.6715464514966379, 6: 0.5001450370818181, 7: 0.2822393064383389, 8: 0.9402458315878417, 9: 0.2657905405040588}, 0)
({0: 0.41429299227040856, 1: 0.40059196758918525, 2: 0.621787171747695, 3: 0.9122077031232282, 4: 0.6161816448475379, 5: 0.9673529227812119, 6: 0.24849079855787165, 7: 0.22698910907704506, 8: 0.6177538662843673, 9: 0.32919829542592816}, 0)
({0: 0.033234076182812905, 1: 0.5225248193493289, 2:

In [ ]:
model = tree.HoeffdingTreeClassifier(    grace_period=100,    delta=1e-5)

In [ ]:
metric = metrics.Accuracy()
evaluate.progressive_val_score(led_stream,model,metric)

Accuracy: 10.71%